# Day 1

In [8]:
from itertools import pairwise, tee
from pathlib import Path

INPUTS = Path("aoc2021_inputs")

# With lazy iterator:
count = 0
with open(INPUTS / "day1.txt", "r") as f:
    for prev_, next_ in pairwise(map(int, f)):
        count += next_ > prev_
assert count == 1292

def triplewise(it):
    a, b, c = tee(it, 3)
    next(b); next(c); next(c)
    yield from zip(a, b, c)
    
def nwise(it, n):
    its = tee(it, n)
    for i, it in enumerate(its):
        for _ in range(i):
            next(it)
    yield from zip(*its)
    
def triplewise_(it):
    yield from nwise(it, 3)

count = 0
with open(INPUTS / "day1.txt", "r") as f:
    sum_prev = float("+inf")
    for v1, v2, v3 in triplewise_(map(int, f)):
        sum_ = v1 + v2 + v3
        count += sum_ > sum_prev
        sum_prev = sum_
assert count == 1262

# Day 2

In [2]:
from pathlib import Path

INPUT_PATH = Path("aoc2021_inputs/day2.txt")

with open(INPUT_PATH, "r") as f:
    moves = f.readlines()

h, v = 0, 0
for line in moves:
    move, size = line.split()

    if move == "forward":
        h += int(size)
    elif move == "up":
        v -= int(size)
    else:
        v += int(size)
        
print(h, v, h * v)
assert h * v == 1727835

1905 907 1727835


In [3]:
from pathlib import Path

INPUT_PATH = Path("aoc2021_inputs/day2.txt")

with open(INPUT_PATH, "r") as f:
    moves = f.readlines()
    
horiz, depth, aim = 0, 0, 0
for line in moves:
    move, size = line.split()
    size = int(size)
    
    if move == "forward":
        horiz += size
        depth += size * aim
    elif move == "down":
        aim += size
    else:
        aim -= size
        
print(horiz, depth, aim, horiz * depth)

1905 810499 907 1544000595


# Day 3

In [39]:
from collections import Counter
from pathlib import Path

INPUT_PATH = Path("aoc2021_inputs/day3.txt")

counter = Counter()
with open(INPUT_PATH, "r") as f:
    for line in f:
        counter += Counter(enumerate(line.strip()))

gamma, epsilon = 0, 0
total_bits = len(counter) // 2
for n in range(total_bits):
    if counter[(n, "1")] > counter[(n, "0")]:
        gamma |= 1 << (total_bits - n - 1)
    else:
        epsilon |= 1 << (total_bits - n - 1)
print(gamma * epsilon)

# --- part 2 ---

with open(INPUT_PATH, "r") as f:
    lines = f.readlines()

under_consideration = lines
prefix = ""
while len(under_consideration) > 1:
    c = Counter(line[:len(prefix) + 1] for line in under_consideration)
    c[prefix + "1"] += 0.5  # Force ending in "1" to win in case of draw.
    (prefix, count), = c.most_common(1)
    under_consideration = [line for line in under_consideration if line.startswith(prefix)]
oxygen = int(under_consideration[0], 2)
print(oxygen)

under_consideration = lines
prefix = ""
while len(under_consideration) > 1:
    c = Counter(line[:len(prefix) + 1] for line in under_consideration)
    c[prefix + "0"] -= 0.5  # Force ending in "0" to lose in case of draw
    _, (prefix, count) = c.most_common(2)
    under_consideration = [line for line in under_consideration if line.startswith(prefix)]
co2 = int(under_consideration[0], 2)
print(co2)

print(oxygen * co2)

749376
3871
613
2372923


# Day 4

In [3]:
from pathlib import Path

INPUT_PATH = Path("aoc2021_inputs/day4.txt")

with open(INPUT_PATH, "r") as f:
    nums = next(f).strip().split(",")  # Read order in which numbers are drawn.
    data = f.read().strip()

# Build dictionary storing the position of each number.
num_pos = dict(zip(nums, range(len(nums))))

# `boards` is a list of pairs (dirs, nums)
# `dirs` contains a list of all the possible winning directions for a board;
# `nums` is the set of all the numbers that show up in that board.
boards = []
for subdata in data.split("\n\n"):
    # Store the rows and columns in the same list.
    directions = []
    # Set to keep track of all the numbers the card contains.
    numbers = set()
    for row in subdata.split("\n"):
        directions.append(row.split())
        # Update the numbers with the ones of the current row.
        numbers |= {*directions[-1]}
    # Add columns to the directions list.
    bs = len(directions)
    directions += [
        [directions[r][c] for r in range(bs)] for c in range(bs)
    ]

    boards.append((directions, numbers))

# Go over each board and figure out when that board is done.
done_at = [
    min(max(num_pos[n] for n in dir_) for dir_ in dirs)
    for dirs, _ in boards
]

def score_board(winning_move_idx):
    _, board_nums = boards[done_at.index(winning_move_idx)]
    unmarked_numbers = map(int, board_nums - set(nums[:winning_move_idx + 1]))
    return sum(unmarked_numbers) * int(nums[winning_move_idx])

print(score_board(min(done_at))) # part 1
print(score_board(max(done_at))) # part 2

27027
36975


# Day 5

In [39]:
from collections import Counter
from pathlib import Path

INPUT_PATH = Path("aoc2021_inputs/day5.txt")

def parse_line(string):
    return [list(map(int, point.split(","))) for point in string.split(" -> ")]

def sign(x):
    return (x > 0) - (x < 0)

def generate_segment(left, right):
    (xl, yl), (xr, yr) = left, right
    delta = max(abs(xl - xr), abs(yl - yr))
    dx, dy = sign(xr - xl), sign(yr - yl)
    return [(xl + dx * d, yl + dy * d) for d in range(delta + 1)]

with open(INPUT_PATH, "r") as f:
    points = [parse_line(line.strip()) for line in f]

counter = Counter()
for left, right in points:
    (xl, yl), (xr, yr) = left, right
    if xl != xr and yl != yr:
        continue
    counter += Counter(generate_segment(left, right))

print(sum(cnt > 1 for _, cnt in counter.items()))

counter = Counter()
for left, right in points:
    counter += Counter(generate_segment(left, right))

print(sum(cnt > 1 for _, cnt in counter.items()))

7142
20012


# Day 6

In [1]:
from collections import Counter
from pathlib import Path

INPUT_PATH = Path("aoc2021_inputs/day6.txt")

with open(INPUT_PATH, "r") as f:
    lanternfish = [int(num) for num in f.readline().split(",")]
    
population = Counter(lanternfish)
sizes = [population.total()]
for cycle in [80, 256 - 80]:
    for _ in range(cycle):
        next_gen = Counter()
        for age in range(8, 0, -1):
            next_gen[age - 1] = population[age]
        next_gen[8] = population[0]
        next_gen[6] += population[0]
        population = next_gen
        sizes.append(population.total())
    print(population.total())  # New in Python 3.10

345387
1574445493136


# Day 7

In [22]:
from math import ceil
from pathlib import Path
from statistics import mean, median

INPUT_PATH = Path("aoc2021_inputs/day7.txt")

with open(INPUT_PATH, "r") as f:
    crabs = [int(num) for num in f.readline().split(",")]

linear_alignment = round(median(crabs))
print(sum(abs(linear_alignment - crab) for crab in crabs))
quad_alignment_options = [c := ceil(mean(crabs)), c - 1]
print(min(sum(
    (d := abs(quad_alignment - crab)) * (d + 1) // 2
     for crab in crabs
) for quad_alignment in quad_alignment_options))

349812
99763899


# Day 8

In [20]:
from collections import Counter
from functools import reduce
from pathlib import Path

INPUT_PATH = Path("aoc2021_inputs/day8.txt")

segment_lengths = [6, 2, 5, 5, 4, 5, 6, 3, 7, 6]

def with_length(clues, l):
    return {s for s in clues if len(s) == l}

def comp(clue):
    return frozenset("abcdefg") - clue

def solve_clues(clues):
    one = with_length(clues, 2).pop()
    four = with_length(clues, 4).pop()
    seven = with_length(clues, 3).pop()
    eight = with_length(clues, 7).pop()

    zero_six_nine = with_length(clues, 6)
    six = [s for s in zero_six_nine if comp(s) < one][0]
    zero_nine = zero_six_nine - {six}
    zero = [s for s in zero_nine if comp(s) < four][0]
    nine = (zero_nine - {zero}).pop()
    
    two_three_five = with_length(clues, 5)
    three = [s for s in two_three_five if one < s][0]
    two_five = two_three_five - {three}
    five = [s for s in two_five if s < nine][0]
    two = (two_five - {five}).pop()
    return [zero, one, two, three, four, five, six, seven, eight, nine]
    

counter = Counter()
with open(INPUT_PATH, "r") as f:
    for line in f:
        _, digits = line.split(" | ")
        counter.update(map(len, digits.strip().split()))
print(sum(counter[segment_lengths[d]] for d in [1, 4, 7, 8]))

outs = []
with open(INPUT_PATH, "r") as f:
    for line in f:
        clue_strings, digits = line.split(" | ")
        clues = [frozenset(string) for string in clue_strings.split()]
        solutions = solve_clues(clues)
        digits = [solutions.index(frozenset(string)) for string in digits.strip().split()]
        outs.append(reduce(lambda l, r: 10 * l + r, digits))
print(sum(outs))

264
1063760


# Day 9

In [15]:
from math import prod
from pathlib import Path

def neighbouring_positions(matrix, r, c):
    return {
        (r_, c_) for r_, c_ in [(r - 1, c), (r, c - 1), (r + 1, c), (r, c + 1)]
        if 0 <= r_ < len(matrix) and 0 <= c_ < len(matrix[0])
    }

def get_neighbours(matrix, r, c):
    to_try = neighbouring_positions(matrix, r, c)
    return {matrix[r_][c_] for r_, c_ in to_try}

def find_basin(matrix, r, c):
    to_visit, visited, basin = {(r, c)}, set(), set()
    while to_visit:
        r_, c_ = to_visit.pop()
        if matrix[r_][c_] == 9:
            continue
        basin.add((r_, c_))
        next_neighbours = neighbouring_positions(matrix, r_, c_)
        to_visit.update(next_neighbours - visited)
        visited.update(next_neighbours)
    return basin

INPUT_PATH = Path("aoc2021_inputs/day9.txt")

with open(INPUT_PATH, "r") as f:
    matrix = [
        [int(num) for num in line.strip()]
        for line in f.readlines()
    ]

risk = 0
low_point_locations = []
for r, row in enumerate(matrix):
    for c, val in enumerate(row):
        neighbs = get_neighbours(matrix, r, c)
        if min(neighbs) > val:
            risk += val + 1
            low_point_locations.append((r, c))
print(risk)

basin_sizes = []
for low_point in low_point_locations:
    basin = find_basin(matrix, *low_point)
    basin_sizes.append(len(basin))
print(prod(sorted(basin_sizes)[-3:]))

502
1330560


# Day 10

In [8]:
from functools import reduce
from pathlib import Path
from statistics import median

OPENING = {"(", "[", "{", "<"}  # Set with opening characters.
CLOSING = {")": "(", "]": "[", "}": "{", ">": "<"}  # Dict with opening character for each closing.
CLOSE_WITH = {v: k for k, v in CLOSING.items()}     # Dict with closing character for each opening.
CORRUPTED_SCORES = {")": 3, "]": 57, "}": 1197, ">": 25137}  # Scores.
COMPLETION_SCORES = {")": 1, "]": 2, "}": 3, ">": 4}

def from_digits(digits, base):
    return reduce(lambda l, r: l * base + r, digits, 0)

INPUT_PATH = Path("aoc2021_inputs/day10.txt")

with open(INPUT_PATH, "r") as f:
    lines = f.readlines()

corrupted_scores = 0
completion_scores = []
for line in lines:
    stack = []
    for char in line:
        if char in OPENING:
            stack.append(char)
        elif char in CLOSING:
            if stack[-1] == CLOSING[char]:
                stack.pop()
            else:
                corrupted_scores += CORRUPTED_SCORES[char]
                break
    else:
        digits = [COMPLETION_SCORES[CLOSE_WITH[char]] for char in reversed(stack)]
        completion_scores.append(from_digits(digits, 5))
print(corrupted_scores)
print(median(completion_scores))

370407
3249889609


# Day 11

In [13]:
from collections import defaultdict
from pathlib import Path

INPUT_PATH = Path("aoc2021_inputs/day11.txt")

grid = defaultdict(lambda: float("-inf"))
with open(INPUT_PATH, "r") as f:
    grid |= {
        (r, c): int(val)
        for r, line in enumerate(f) for c, val in enumerate(line.strip())
    }

def neighbouring_positions(r, c):
    return [
        (r + 1, c), (r + 1, c + 1), (r, c + 1), (r - 1, c + 1),
        (r - 1, c), (r - 1, c - 1), (r, c - 1), (r + 1, c - 1),
    ]

flashes, flashed, step = 0, set(), 0
while len(flashed) < 100:
    step += 1
    flashed = set()
    to_increment = [(r, c) for r in range(10) for c in range(10)]
    while to_increment:
        r, c = to_increment.pop()
        if not ((0 <= r < 10) and (0 <= c < 10)):
            continue
        grid[r, c] += 1
        if grid[r, c] > 9 and (r, c) not in flashed:
            flashes += 1
            flashed.add((r, c))
            to_increment.extend(neighbouring_positions(r, c))
    for r in range(10):
        for c in range(10):
            grid[r, c] = 0 if grid[r, c] > 9 else grid[r, c]
    if step == 100:
        print(flashes)
print(step)

1585
382


# Day 12

In [15]:
from collections import defaultdict
from pathlib import Path

INPUT_PATH = Path("aoc2021_inputs/day12.txt")

def count_paths(graph, path_built, can_revisit=True):
    if path_built and path_built[-1] == "end":
        return 1

    return sum(
        count_paths(
            graph,
            path_built + [next_stop],
            can_revisit and not (next_stop.islower() and next_stop in path_built),
        )
        for next_stop in graph[path_built[-1]]
        if next_stop not in path_built or next_stop.isupper() or can_revisit
    )

with open(INPUT_PATH, "r") as f:
    lines = f.readlines()

graph = defaultdict(list)
for line in lines:
    pt1, pt2 = line.strip().split("-")
    graph[pt1].append(pt2)
    graph[pt2].append(pt1)
# Make sure no cave points back at the “start” cave.
for connections in graph.values():
    try:
        connections.remove("start")
    except ValueError:
        pass

print(count_paths(graph, ["start"], False))
print(count_paths(graph, ["start"], True))

3410
98796


# Day 13

In [2]:
from itertools import chain
from pathlib import Path

INPUT_PATH = Path("aoc2021_inputs/day13.txt")

with open(INPUT_PATH, "r") as f:
    contents = f.read()

def fold(points, fold_point):
    xf, yf = fold_point
    return {
        (x if x < xf else 2 * xf - x, y if y < yf else 2 * yf - y)
        for x, y in points
    }

point_data, fold_data = contents.split("\n\n")
points = {tuple(map(int, line.split(","))) for line in point_data.splitlines()}
folds = fold_data.splitlines()

MAX_COORD = 1 + max(chain.from_iterable(points))

for fold_string in folds:
    coord, value = fold_string.removeprefix("fold along ").split("=")
    fold_point = (
        int(value) if coord == "x" else MAX_COORD,
        int(value) if coord == "y" else MAX_COORD,
    )
    points = fold(points, fold_point)

width = max(x for x, _ in points) + 1
height = max(y for _, y in points) + 1
result = [[" "] * width for _ in range(height)]
for x, y in points:
    result[y][x] = "\u2588"
print("\n".join("".join(line) for line in result))

████  ██  █  █  ██  █  █ ███  ████  ██ 
█    █  █ █ █  █  █ █ █  █  █    █ █  █
███  █    ██   █    ██   ███    █  █   
█    █ ██ █ █  █    █ █  █  █  █   █ ██
█    █  █ █ █  █  █ █ █  █  █ █    █  █
█     ███ █  █  ██  █  █ ███  ████  ███


# Day 14

In [28]:
from collections import Counter
from itertools import chain
from math import ceil
from pathlib import Path

try:
    from itertools import pairwise  # Python 3.10
except ImportError:
    from itertools import tee
    def pairwise(it):
        it1, it2 = tee(it, 2)
        next(it2)
        yield from zip(it1, it2)

INPUT_PATH = Path("aoc2021_inputs/day14.txt")

with open(INPUT_PATH, "r") as f:
    polymer = f.readline().strip()
    f.readline()  # empty line
    rules = dict(line.strip().split(" -> ") for line in f)

counter = Counter("".join(item) for item in pairwise(polymer))

for _ in range(40):
    new_counter = Counter()
    for (char1, char2), count in counter.items():
        target = rules[char1 + char2]
        new_counter[char1 + target] += count
        new_counter[target + char2] += count
    counter = new_counter

element_counter = Counter()
for (char1, char2), count in counter.items():
    element_counter[char1] += count
    element_counter[char2] += count
print(ceil(max(element_counter.values()) / 2) - ceil(min(element_counter.values()) / 2))

3015383850689


In [26]:
polymer

'FNFPPNKPPHSOKFFHOFOC'

# Day 15

In [3]:
from queue import PriorityQueue
from pathlib import Path

INPUT_PATH = Path("aoc2021_inputs/day15.txt")

def neighbouring_costs(grid, r, c):
    return [(grid.get((r_, c_), float("+inf")), (r_, c_)) for r_, c_ in [(r + 1, c), (r - 1, c), (r, c + 1), (r, c - 1)]]

with open(INPUT_PATH, "r") as f:
    grid = {(r, c): int(n) for r, line in enumerate(f) for c, n in enumerate(line.strip())}

def dijkstra(costs, from_, dest_):
    visited = {from_}
    q = PriorityQueue()
    q.put((0, from_))
    while True:
        cost, current_node = q.get()
        if current_node == dest_:
            return cost
        for tot, pos in neighbouring_costs(costs, *current_node):
            if tot < float("inf") and pos not in visited:
                visited.add(pos)
                q.put((cost + tot, pos))

print(dijkstra(grid, (0, 0), (99, 99)))

R, C = 1 + max(t[0] for t in grid.keys()), 1 + max(t[1] for t in grid.keys())
larger_map = {
    (r, c): ((grid[r % R, c % C] - 1 + (r//R) + (c//C)) % 9) + 1 for c in range(5 * C) for r in range(5 * R)
}

from time import time
start = time()
print(dijkstra(larger_map, (0,0), (5 * R - 1, 5 * C - 1)))
print(time() - start)

508
2872
2.1289970874786377


# Day 16

In [68]:
"""
BNF-ish grammar that parses the packets.
{n} is used to indicate "repeat n times".

A literal packet is a tuple (type, version, value)
An operator packet is a tuple (type, version, flag, list_of_packets)

packet := type (version="100" value | version operator_packets)
value := ("1" group)* "0" group
group := D{4}
# (Bit) Length of packets read is specified by `bit_length`.
# Number of packets read is specified by `packet_length`.
operator_packets := ("0" bit_length packet* | "1" packet_length packet{packet_length})
bit_length := D{15}
packet_length := D{11}
type := D{3}
version := D{3}
D := "1" | "0"
"""

from functools import partial
from itertools import chain
from math import prod
from pathlib import Path

INPUT_PATH = Path("aoc2021_inputs/day16.txt")

def parse(hexadecimal_packet):
    packet_string = "".join(
        f"{int(letter, 16):04b}" for letter in hexadecimal_packet
    )
    return parse_packet(packet_string)

def parse_packet(packet_string):
    type_, packet_string = parse_type(packet_string)
    version, packet_string = parse_type(packet_string)
    if version == "100":
        value, packet_string = parse_value(packet_string)
        return (int(type_, 2), int(version, 2), int(value, 2)), packet_string
    else:
        (flag, packets), packet_string = parse_operator_packets(packet_string)
        return (int(type_, 2), int(version, 2), int(flag), packets), packet_string

def parse_value(packet_string):
    parsed = ""
    while packet_string[0] == "1":
        group, packet_string = parse_group(packet_string[1:])
        parsed += group
    expect(packet_string[0], "0")
    group, packet_string = parse_group(packet_string[1:])
    return parsed + group, packet_string

def parse_operator_packets(packet_string):
    flag, packet_string = parse_D(packet_string)
    if flag == "0":
        length, packet_string = parse_bit_length(packet_string)
        bit_length = int(length, 2)
        to_parse = packet_string[:bit_length]  # Part of the string to be parsed.
        packets = []
        while to_parse:
            packet, to_parse = parse_packet(to_parse)
            packets.append(packet)
        packet_string = packet_string[bit_length:]
    elif flag == "1":
        length, packet_string = parse_packet_length(packet_string)
        packet_number = int(length, 2)
        packets = []
        for _ in range(packet_number):
            packet, packet_string = parse_packet(packet_string)
            packets.append(packet)
    return (flag, packets), packet_string

### === Basic parsers === ###
def parse_Dn(n, packet_string):
    parsed = ""
    for _ in range(n):
        D, packet_string = parse_D(packet_string)
        parsed += D
    return parsed, packet_string

def parse_D(packet_string):
    expect(packet_string[0], "01")
    return packet_string[0], packet_string[1:]

### --- helper function --- ###
def expect(D, values):
    if D not in values:
        raise ValueError(f"Expected one of {values}.")

def sum_versions(packet):
    sub = 0 if len(packet) == 3 else sum(sum_versions(sub) for sub in packet[3])
    return packet[0] + sub

parse_group = partial(parse_Dn, 4)
parse_bit_length = partial(parse_Dn, 15)
parse_packet_length = partial(parse_Dn, 11)
parse_type = partial(parse_Dn, 3)
parse_version = partial(parse_Dn, 3)

with open(INPUT_PATH, "r") as f:
    hexadecimal_string = "".join(f.read().strip().splitlines())

transmission, _ = parse(hexadecimal_string)
#transmission, _ = parse("A0016C880162017C3686B18A3D4780")

print(sum_versions(transmission))

eval_dict = {
    0: sum,
    1: prod,
    2: min,
    3: max,
    5: lambda packets: packets[0] > packets[1],
    6: lambda packets: packets[0] < packets[1],
    7: lambda packets: packets[0] == packets[1],
}

def evaluate(packet):
    if len(packet) == 3:
        return packet[2]

    subs = [evaluate(sub) for sub in packet[3]]
    return eval_dict[packet[1]](subs)

print(evaluate(transmission))

947
660797830937


# Day 17

In [87]:
from itertools import count
from math import ceil, floor
from pathlib import Path

INPUT_PATH = Path("aoc2021_inputs/day17.txt")

def xpos(v0, steps):
    delta = min(steps, v0)
    return delta * v0 - delta * (delta - 1) // 2

def ypos(v0, steps):
    return steps * v0 - steps * (steps - 1) // 2

with open(INPUT_PATH, "r") as f:
    xdata, ydata = f.readline().strip().removeprefix("target area: ").split(", ")

x_min, x_max = map(int, xdata.removeprefix("x=").split(".."))
y_min, y_max = map(int, ydata.removeprefix("y=").split(".."))

highest = float("-inf")
# Iterate over possible values of vx (velocity along the x axis).
count = 0
for vx in range(x_max, 0, -1):
    for vy in range(min(y_min, 0), 1 + max(abs(y_max), abs(y_min))):
        positions = []
        on_target = False
        while (not positions) or (positions[-1][1] <= x_max and positions[-1][0] >= y_min):
            x, y = xpos(vx, len(positions)), ypos(vy, len(positions))
            on_target = on_target or x_min <= x <= x_max and y_min <= y <= y_max
            positions.append((y, x))
        if on_target:
            highest = max(highest, max(positions)[0])
        count += on_target

print(highest)
print(count)

4186
2709


# Day 18

In [77]:
from functools import reduce
from itertools import product
from math import ceil, floor
from pathlib import Path

INPUT_PATH = Path("aoc2021_inputs/day18.txt")

def snail_read(snail):
    """Reads a snail that is assumed to only have 1-digit values."""
    values, depths, depth = [], [], 0
    for char in snail:
        if char == "[":
            depth += 1
        elif char == "]":
            depth -= 1
        elif char == ",":
            depths.append(depth)
        elif char in "0123456789":
            values.append(int(char))
    return values, depths

def snail_print(snail):
    """Takes a snail and returns its string representation."""
    values, depths = snail
    result, prev_depth = "", 0
    for idx, depth in enumerate(depths):
        if depth > prev_depth:
            result += "[" * (depth - prev_depth) + str(values[idx]) + ","
        else:
            result += str(values[idx]) + "]" * (prev_depth - depth) + ","
        prev_depth = depth
    result += str(values[-1]) + "]" * prev_depth
    return result

def snail_add(left, right):
    """Adds two snails and returns the result in reduced form."""
    (lvals, ldepths), (rvals, rdepths) = left, right
    snail = (lvals + rvals, [1 + d for d in ldepths] + [1] + [1 + d for d in rdepths])
    while True:
        snail, flag = snail_explode(snail)
        if flag:
            continue
        snail, flag = snail_split(snail)
        if not flag:
            break
    return snail

def snail_explode(snail):
    values, depths = snail
    try:
        idx = depths.index(5)
    except ValueError:
        return snail, False

    new_values = values[::]
    if idx > 0:
        new_values[idx - 1] += values[idx]
    if idx + 2 < len(new_values):
        new_values[idx + 2] += values[idx + 1]
    new_values = new_values[:idx] + [0] + new_values[idx + 2:]
    depths = depths[:idx] + depths[idx + 1:]
    return (new_values, depths), True

def snail_split(snail):
    values, depths = snail
    if all((witness := val) < 10 for val in values):
        return snail, False

    idx = values.index(witness)
    new_depth = 1 + max(
        depths[idx - 1] if idx > 0 else 0,
        depths[idx] if idx < len(depths) else 0,
    )
    values = values[:idx] + [floor(witness / 2), ceil(witness / 2)] + values[idx + 1:]
    depths = depths[:idx] + [new_depth] + depths[idx:]
    return (values, depths), True

def snail_magnitude(snail):
    def combine(stack):
        stack.append(2 * stack.pop() + 3 * stack.pop())

    values, depths = snail
    prev_depth, sub_magnitudes = 0, []
    for val, dep in zip(values, depths):
        sub_magnitudes.append(val)
        for _ in range(prev_depth - dep):
            combine(sub_magnitudes)
        prev_depth = dep
    sub_magnitudes.append(values[-1])
    for _ in range(prev_depth):
        combine(sub_magnitudes)
    return sub_magnitudes[0]

with open(INPUT_PATH, "r") as f:
    snails = [snail_read(line.strip()) for line in f]

print(
    snail_magnitude(reduce(snail_add, snails))
)
print(max(
    snail_magnitude(snail_add(left, right)) for left, right in product(snails, repeat=2) if left != right
))

3806
4727


# Day 19

In [5]:
# https://www.reddit.com/r/adventofcode/comments/rjwhdv/2021_day19_i_need_help_understanding_how_to_solve/

# To read later:
#  - https://www.reddit.com/r/adventofcode/comments/rjpf7f/comment/hp87fkd/?utm_source=share&utm_medium=web2x&context=3
#  - https://www.reddit.com/r/adventofcode/comments/rjpf7f/comment/hp86e4k/?utm_source=share&utm_medium=web2x&context=3
#  - https://www.reddit.com/r/adventofcode/comments/rjpf7f/comment/hp8599l/?utm_source=share&utm_medium=web2x&context=3

from functools import cache
from itertools import product
from pathlib import Path

INPUT_PATH = Path("aoc2021_inputs/day19.txt")
MIN_MATCH = 12
ROTATIONS = [  # There has to be a nice way of building this decently.
    # facing positive x
    (1, 2, 3),
    (1, -3, 2),
    (1, -2, -3),
    (1, 3, -2),
    # facing negative x
    (-1, -2, 3),
    (-1, 3, 2),
    (-1, 2, -3),
    (-1, -3, -2),
    # facing positive y
    (2, 3, 1),
    (2, -1, 3),
    (2, -3, -1),
    (2, 1, -3),
    # facing negative y
    (-2, -3, 1),
    (-2, 1, 3),
    (-2, 3, -1),
    (-2, -1, -3),
    # facing positive z
    (3, 1, 2),
    (3, -2, 1),
    (3, -1, -2),
    (3, 2, -1),
    # facing negative z
    (-3, -1, 2),
    (-3, 2, 1),
    (-3, 1, -2),
    (-3, -2, -1),
]

def sign(x):
    return (x > 0) - (x < 0)

class Point:
    """Class to represent 2D or 3D points."""
    def __init__(self, x, y, z=0):
        self.x, self.y, self.z = x, y, z

    @classmethod
    def fromstring(cls, string):
        """Create a point from a comma-separated string."""
        return cls(*map(int, string.strip().split(",")))

    def __add__(self, other):
        """Adds a point to another point or to a scalar number."""
        if isinstance(other, Point):
            return Point(self.x + other.x, self.y + other.y, self.z + other.z)
        else:
            return Point(self.x + other, self.y + other, self.z + other)

    def __sub__(self, other):
        """Subtracts a point from another point or scalar number."""
        return self + (-other)

    def __mul__(self, other):
        """Multiplies a point with a number."""
        return Point(self.x * other, self.y * other, self.z * other)

    def __rmul__(self, other):
        return self * other

    def __neg__(self):
        """Negates a point."""
        return Point(-self.x, -self.y, -self.z)

    def __abs__(self):
        """Returns the sum of the absolute values of the coordinates."""
        return abs(self.x) + abs(self.y) + abs(self.z)

    def __eq__(self, other):
        """Checks if a point is equal to another point."""
        return isinstance(other, Point) and (
            self.x == other.x and
            self.y == other.y and
            self.z == other.z
        )

    def __repr__(self):
        return f"<{self.x}, {self.y}, {self.z}>"

    def __hash__(self):
        """Gives a unique hash to each point to allow points to be contained in sets."""
        return hash((self.x, self.y, self.z))

    def __iter__(self):
        return iter((self.x, self.y, self.z))

    def rotate(self, rot):
        coords = [None, self.x, self.y, self.z]
        return Point(*[sign(c) * coords[abs(c)] for c in rot])


def to_edges(beacons):
    """Turns a list of beacons into a list of lists of edges."""
    return [
        [y - x for y in beacons]
        for x in beacons
    ]


def orientations(edges):
    """Yield the 24 possible rotations of the list of edges."""
    for rot in ROTATIONS:
        yield [edge.rotate(rot) for edge in edges]

'''
def match(reference, others, threshold):
    """Check if the two structures match geometrically.
    
    `reference` is a list of edges, and `others` is a list of lists of edges,
    seen from the perspective of each point.
    A match is defined by an overlap of `threshold` or more points,
    which means `threshold - 1` or more edges.
    """

    for idx, oth in enumerate(others):
        if len(set(reference) & set(oth)) >= threshold - 1:
            return idx
    return None
'''


def match(references, others, threshold):
    """Check if the two structures match geometrically.

    `references` is a list of lists of edges and `others` is a list of lists of edges.
    A match is defined by an overlap of `threshold` or more points,
    which means `threshold - 1` or more edges.
    """

    it = product(enumerate(references[:-(threshold - 1)]), enumerate(others))
    for (r_idx, ref), (o_idx, oth) in it:
        if len(set(ref) & set(oth)) >= threshold - 1:
            return True, r_idx, o_idx
    return False, None, None


def transform(reference, others, threshold):
    """Given two lists of points in 3D, try to transform `others`
    into the same coordinate system as `reference`.
    """

    ref_edges = to_edges(reference)
    # Try all possible orientations of the other points.
    for orientation in orientations(others):
        other_edges = to_edges(orientation)
        flag, r_idx, o_idx = match(ref_edges, other_edges, threshold)
        if not flag:
            continue
        # If flag is `True`, then reference[r_idx] and orientation[o_idx] match.
        vec = reference[r_idx] - orientation[o_idx]
        return True, reference, [other + vec for other in orientation], vec
    return False, reference, others, None


with open(INPUT_PATH, "r") as f:
    scanners = [
        tuple(map(Point.fromstring, scanner.splitlines()[1:]))
        for scanner in f.read().split("\n\n")
    ]

scanner_positions = [Point(0, 0, 0) for _ in scanners]
to_do = set(range(len(scanners)))
no_changes = False
while len(to_do) > 1 and not no_changes:
    no_changes = True
    for ref, oth in product(to_do, to_do):
        if ref == oth or len(scanners[ref]) > len(scanners[ref]):
            continue
        flag, ref_points, oth_points, vec = transform(scanners[ref], scanners[oth], MIN_MATCH)
        if flag:
            print(f"{oth:2} -> {ref:2}")
            to_do.remove(oth)
            scanners[ref] = tuple(set(ref_points) | set(oth_points))
            scanner_positions[oth] -= vec
            no_changes = False
            break


assert not no_changes, "The algo. failed."
final = to_do.pop()
print(len(scanners[final]))
print(max(
    abs(x - y) for x, y in product(scanner_positions, repeat=2)
))

24 ->  0
16 ->  0
 5 ->  0
18 ->  0
 2 ->  0
13 ->  0
 3 ->  0
12 ->  0
 8 ->  0
 9 ->  0
14 ->  0
15 ->  0
17 ->  0
10 ->  0
 1 ->  0
 6 ->  0
11 ->  0
 7 ->  0
19 ->  0
20 ->  0
21 ->  0
 4 ->  0
22 ->  0
23 ->  0
303
9621


# Day 20

In [43]:
from functools import reduce
from itertools import product
from pathlib import Path

INPUT_PATH = Path("aoc2021_inputs/day20.txt")


def get_next(image, pos, algorithm, padding):
    x, y = pos
    binary = "".join(
        image.get((x + dx, y + dy), padding)
        for dy, dx in product((-1, 0, 1), repeat=2)
    )
    return algorithm[int(binary, 2)]


def enhance(algorithm, image, bounds, padding):
    (x_min, x_max), (y_min, y_max) = bounds
    img = {}
    for pos in product(range(x_min - 1, x_max + 2), range(y_min - 1, y_max + 2)):
        img[pos] = get_next(image, pos, algorithm, padding)

    new_bounds = ((x_min - 1, x_max + 1), (y_min - 1, y_max +1))
    new_padding = algorithm[0] if padding == "0" else algorithm[-1]
    return img, new_bounds, new_padding


with open(INPUT_PATH, "r") as f:
    algo = ["1" if val == "#" else "0" for val in f.readline().strip()]
    f.readline()
    image = {
        (x, y): "1" if val == "#" else "0"
        for y, line in enumerate(f) for x, val in enumerate(line.strip())
    }

max_x, max_y = max(image.keys())
bounds = ((0, max_x), (0, max_y))
padding = "0"

# for _ in range(2):
for _ in range(50):
    image, bounds, padding = enhance(algo, image, bounds, padding)
sum(val == "1" for val in image.values())

18131

# Day 21

In [84]:
from itertools import cycle, islice
from pathlib import Path

INPUT_PATH = Path("aoc2021_inputs/day21.txt")
ROLLS = 3
SIDES = 100

die = cycle(range(1, SIDES + 1))

with open(INPUT_PATH, "r") as f:
    players_start = [int(line.strip().split()[-1]) for line in f]

seed = list(range(1, 11))
positions = [cycle(seed[p:] + seed[:p]) for p in players_start]
scores = [0] * len(positions)

p, n = 0, len(positions)
while max(scores) < 1000:
    positions[p % n] = islice(positions[p % n], sum(islice(die, ROLLS)) - 1, None)
    scores[p % n] += next(positions[p % n])
    p += 1
print(p * ROLLS * min(scores))

576600


In [132]:
from collections import Counter
from functools import cache
from itertools import product

advances = Counter(sum(perm) for perm in product(range(1, 4), repeat=3))

@cache
def play(pos, scores, player):
    # print(f"Playing {pos = }, {scores = }, {player = }.")
    pos, scores = list(pos), list(scores)
    if max(scores) >= 21:
        return [int(sc >= 21) for sc in scores]

    outcomes = [0] * len(pos)
    for roll, count in advances.items():
        pos_, scores_ = pos[::], scores[::]
        pos_[player] = (pos_[player] - 1 + roll) % 10 + 1
        scores_[player] += pos_[player]
        outcomes = [
            out + count * win
            for out, win in zip(outcomes, play(tuple(pos_), tuple(scores_), 1 - player))
        ]
    return outcomes

In [138]:
max(play(tuple(players_start), (0, 0), 0))

131888061854776

# Day 22

## Part 1 (brute-force)

In [5]:
from collections import defaultdict
from itertools import product
from pathlib import Path

INPUT_PATH = Path("aoc2021_inputs/day22.txt")

with open(INPUT_PATH, "r") as f:
    steps = [line.strip() for line in f.readlines()[:20]]
rects = [
    [tuple(map(int, bit.split("=")[1].split(".."))) for bit in coords.split(",")]
    for coords in steps
]

reactor = defaultdict(int)
for step in steps:
    target, coords = step.split()
    target = target == "on"
    xr, yr, zr = [tuple(map(int, bit.split("=")[1].split(".."))) for bit in coords.split(",")]
    for coords in product(range(xr[0], xr[1] + 1), range(yr[0], yr[1] + 1), range(zr[0], zr[1] + 1)):
        reactor[coords] = target
print(sum(reactor.values()))

568000


## Part 2

In [118]:
from itertools import product, starmap
from math import prod
from queue import SimpleQueue

def split_interval(reference, other):
    """Splits the other interval in pieces in or out of the reference interval."""

    (ref_a, ref_b), (other_a, other_b) = reference, other
    if other_b < ref_a or ref_b < other_a or ref_a <= other_a <= other_b <= ref_b:
        intervals = [other]
    elif other_a <= ref_a <= other_b <= ref_b:
        intervals = [(other_a, ref_a - 1), (ref_a, other_b)]
    elif ref_a <= other_a <= ref_b <= other_b:
        intervals = [(other_a, ref_b), (ref_b + 1, other_b)]
    elif other_a <= ref_a <= ref_b <= other_b:
        intervals = [
            (other_a, ref_a - 1),
            (ref_a, ref_b),
            (ref_b + 1, other_b),
        ]
    return [(a, b) for a, b in intervals if b >= a]

def split(reference, other):
    """Splits the other hypercube in pieces in or out of the reference hypercube."""

    to_handle = SimpleQueue()
    to_handle.put((tuple(), reference, other))
    final = []

    if any(
        oth_a > ref_b or ref_a > oth_b
        for (ref_a, ref_b), (oth_a, oth_b) in zip(reference, other)
    ):
        return [other]

    return list(product(*starmap(split_interval, zip(reference, other))))

def inside(reference, other):
    """Determine whether or not the other hypercube is contained inside the reference.

    Assumes that `other` is either completely contained in, or completely outside of,
    the `reference` hypercube.
    """

    vertices = product(*other)
    return any(
        all(a <= coord <= b for (a, b), coord in zip(reference, vertex))
        for vertex in vertices
    )

def volume(hypercube):
    return prod(b - a + 1 for a, b in hypercube)

In [120]:
from itertools import product
from pathlib import Path
from time import time

INPUT_PATH = Path("aoc2021_inputs/day22.txt")

with open(INPUT_PATH, "r") as f:
    steps = [line.strip() for line in f.readlines()]

rects = [
    tuple([tuple(map(int, bit.split("=")[1].split(".."))) for bit in coords.split(",")])
    for coords in steps
]
instructions = [step.split()[0] for step in steps]

start = time()

# `on` is a list of fully disjoint regions where the lights are on.
on = set()
for rect, instruction in zip(rects, instructions):
    if instruction == "on":
        # Figure out what parts of the region we are handling haven't
        # been turned ON yet.
        pieces = {rect}
        for on_region in on:
            pieces = {
                p
                for piece in pieces for p in split(on_region, piece)
                if not inside(on_region, p)
            }
        on |= pieces
    elif instruction == "off":
        # Figure out what parts of the ON regions are inside the region
        # we are handling right now, and drop those.
        new_on = set()
        for on_region in on:
            new_on.update(
                piece for piece in split(rect, on_region)
                if not inside(rect, piece)
            )
        on = new_on

print(sum(volume(region) for region in on))
print(f"Finished in {(time() - start) / 60:.2f} minutes.")

1177411289280259
Finished in 7.44 minutes.
